In [1]:
import os
from openai import OpenAI

from models import JobDescription, Resume
from job_description_parser import JobDescriptionParser
from resume_parser import ResumeParser
from similarity_score import MatchingEngine


In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
job_description_parser = JobDescriptionParser(llm_client)
resume_parser = ResumeParser(llm_client)
matching_engine = MatchingEngine(llm_client)

In [3]:
def calculate_match_score(job_description_text: str, resume_text: str):
    job_description = job_description_parser.parse(job_description_text)
    resume = resume_parser.parse(resume_text)
    return matching_engine.match(job_description, resume)

### Test Case 1: Perfect Match Scenario

In [4]:
sample_jd_1 = """
Job Title: Software Engineer
Location: San Francisco, CA
Industry/Domain: Technology
Education Degree: Bachelor's in Computer Science or related field
Technical Skills: Python, Java, SQL, Git, Agile methodologies
"""

sample_resume_1 = """
Name: Jane Doe
Location: San Francisco, CA
Education: Bachelor's in Computer Science, University of California, Berkeley
Experience: Software Engineer at TechCorp, proficient in Python, Java, SQL, Git, and experienced in Agile methodologies.
"""

calculate_match_score(sample_jd_1, sample_resume_1)

The job description is as follows: {"job_title": "Software Engineer", "location": "San Francisco, CA", "industry": "Technology", "education_degree": "Bachelor's in Computer Science or related field", "technical_skills": ["Python", "Java", "SQL", "Git", "Agile methodologies"]} and the resume is as follows: {"candidate_name": "Jane Doe", "email": null, "phone": null, "location": "San Francisco, CA", "industry": ["Technology", "Software Engineering"], "education_degree": ["Bachelor's in Computer Science"], "skills": ["Python", "Java", "SQL", "Git", "Agile methodologies"]}


{'similarity_comments': "The resume of Jane Doe matches the job description for a Software Engineer position very closely. The key attributes align well across various categories, leading to a high similarity score. Specifically:\n- Location: Both are based in San Francisco, CA, indicating no relocation would be necessary.\n- Industry: The job is in the Technology sector, and Jane's experience includes Technology and Software Engineering, which is a direct match.\n- Education Degree: The required degree is a Bachelor's in Computer Science or a related field, and Jane has a Bachelor's in Computer Science, meeting the education criteria perfectly.\n- Technical Skills: The job requires Python, Java, SQL, Git, and Agile methodologies, all of which are listed in Jane's resume, showcasing a direct match in required technical skills.\nGiven these matches across all critical attributes, the overall similarity score is deemed very high.",
 'similarity_score': 98}

### Test Case 2: Partial Match Scenario

In [5]:
sample_jd_2 = """
Job Title: Data Analyst
Location: New York, NY
Industry/Domain: Finance
Education Degree: Bachelor's in Statistics or related field
Technical Skills: R, Python, SQL, Excel, Tableau
"""

sample_resume_2 = """
Name: John Smith
Location: Boston, MA
Education: Bachelor's in Mathematics, Boston University
Experience: Data Analyst at FinTech Inc., proficient in R, Python, SQL, and Excel. Familiar with Tableau.
"""

calculate_match_score(sample_jd_2, sample_resume_2)

The job description is as follows: {"job_title": "Data Analyst", "location": "New York, NY", "industry": "Finance", "education_degree": "Bachelor's in Statistics or related field", "technical_skills": ["R", "Python", "SQL", "Excel", "Tableau"]} and the resume is as follows: {"candidate_name": "John Smith", "email": null, "phone": null, "location": "Boston, MA", "industry": ["Data Analysis", "Finance", "Technology"], "education_degree": ["Bachelor's in Mathematics"], "skills": ["R", "Python", "SQL", "Excel", "Tableau"]}


{'similarity_comments': "The job requires a location in New York, NY, but the candidate is based in Boston, MA, which might affect their ability to fulfill location-based requirements. However, the industry match is strong, as the candidate has experience in Finance, which is sought by the job, alongside additional exposure to Data Analysis and Technology. The education requirement almost aligns with the job's specification; while the job specifies Statistics or related field, Mathematics is closely related and should satisfy the requirement. The candidate's skills perfectly match the job's technical requirements, making them a strong candidate in terms of capabilities. Overall, the main discrepancy is the location, but in all other aspects, the candidate matches well with the job description.",
 'similarity_score': 85}

### Test Case 3: No Match Scenario

In [6]:

sample_jd_3 = """
Job Title: Graphic Designer
Location: Los Angeles, CA
Industry/Domain: Media
Education Degree: Bachelor's in Graphic Design or related field
Technical Skills: Adobe Creative Suite, Sketch, InVision, Typography, Branding
"""

sample_resume_3 = """
Name: Emily Johnson
Location: Miami, FL
Education: Bachelor's in Marketing, University of Miami
Experience: Marketing Specialist at Creative Agency, experienced in social media campaigns and content creation. No graphic design experience.
"""

calculate_match_score(sample_jd_3, sample_resume_3)

The job description is as follows: {"job_title": "Graphic Designer", "location": "Los Angeles, CA", "industry": "Media", "education_degree": "Bachelor's in Graphic Design or related field", "technical_skills": ["Adobe Creative Suite", "Sketch", "InVision", "Typography", "Branding"]} and the resume is as follows: {"candidate_name": "Emily Johnson", "email": null, "phone": null, "location": "Miami, FL", "industry": ["Marketing"], "education_degree": ["Bachelor's in Marketing"], "skills": ["social media campaigns", "content creation", "no graphic design experience"]}


{'similarity_comments': "The locations do not match, as the job is in Los Angeles, CA, and the candidate is based in Miami, FL. The industries are somewhat related but not the same, as the job is in the Media industry, while the candidate has experience in Marketing. The education degrees are in related fields but not the same, with the job requiring a Bachelor's in Graphic Design or related field, and the candidate having a Bachelor's in Marketing. Most importantly, the technical skills required for the job, such as Adobe Creative Suite, Sketch, InVision, Typography, and Branding, are not mentioned in the candidate's resume, which lists skills in social media campaigns, content creation, and explicitly states no graphic design experience. This results in a significant disparity in the match for the required technical skills.",
 'similarity_score': 20}